# 03 modeling : naive

- 목적 : 전처리 없이 2depth 기준으로, 토큰나이저로만 모델 학습이 가능한지 확인
- 방법 : 
        1) 사전 학습된 모델을 활용하여 분류기 생성 
    

In [1]:
import pandas as pd
import numpy as np


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [45]:
import glob

In [46]:
file_list =glob.glob('../result/*')

In [47]:
all_df_u = pd.DataFrame()
for file in file_list:
    df = pd.read_csv(file)
    df_u = df.drop_duplicates(subset = ['title'])
    all_df_u = pd.concat([all_df_u, df_u])

In [53]:
all_df_u.categories.nunique()

201

In [54]:
all_df_u['title_desc'] = all_df_u['title'] + ' ' + all_df_u['description']

In [68]:
rawdata = all_df_u[['title_desc','categories']].reset_index()

In [69]:
cate2idx ={k : idx for idx, k in enumerate(rawdata['categories'].unique())}

In [70]:
idx2cate = {v : k for k , v in cate2idx.items()}

In [71]:
rawdata['label'] = rawdata['categories'].map(cate2idx)

In [72]:
import re
pattern = re.compile('[가-힣ㄱ-ㅎA-Za-z0-9.,]+')
rawdata['text'] = rawdata['title_desc'].apply(lambda x: ' '.join(re.findall(pattern, x)))

In [73]:
rawdata.head()

,index,title_desc,categories,label,text
0,0,한국 창작 SF의 거의 모든 것 none,none,0,한국 창작 SF의 거의 모든 것 none
1,1,"상상력을 자극하는 판타지 SF 스토리북 이 책은 3,000페이지에 달하는 총 40편...",Fiction,1,"상상력을 자극하는 판타지 SF 스토리북 이 책은 3,000페이지에 달하는 총 40편..."
2,2,SF 명예의 전당 3 『SF 명예의 전당 3 : 유니버스』는 미국 SF작가협회(SF...,Fiction,1,SF 명예의 전당 3 SF 명예의 전당 3 유니버스 는 미국 SF작가협회 SFWA ...
3,3,"외계 생명체의 증언 <책 소개> 우주선 안에서의 살인 사건, 그리고 유일한 목격자가...",Fiction,1,"외계 생명체의 증언 책 소개 우주선 안에서의 살인 사건, 그리고 유일한 목격자가 외..."
4,4,SF 명예의 전당 4 『SF 명예의 전당 3 : 유니버스』는 미국 SF작가협회(SF...,Fiction,1,SF 명예의 전당 4 SF 명예의 전당 3 유니버스 는 미국 SF작가협회 SFWA ...


In [74]:
rawdata.label.value_counts()

0      582
1      196
4      146
44      83
26      64
      ... 
85       1
84       1
81       1
80       1
200      1
Name: label, Length: 201, dtype: int64

In [75]:
# data arguments

In [76]:
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
m = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(m)
model = AutoModelForSequenceClassification.from_pretrained(m)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [28]:
num_augmentations = 5

In [137]:
def augment_sentence(sentence, model, tokenizer, num_tokens_to_mask=3, num_augmentations = 5):
    augmented_sentences = []
    for _ in range(num_augmentations):
        input_tokens = tokenizer(sentence, 
                                               return_tensors= 'pt',
                                               padding = 'max_length',
                                               max_length = 512,
                                               truncation = True
                                               ).to(device)

        valid_indices = [
                    idx for idx, token_id in enumerate(input_tokens['input_ids'][0])
                    if token_id not in (tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.eos_token_id)
                ]

        token_indices_to_mask = random.sample(range(len(valid_indices)), 1)
        masked_input_tokens = input_tokens.copy()
        masked_input_tokens['input_ids'][0, token_indices_to_mask] = tokenizer.mask_token_id

        with torch.no_grad():
            masked_input_tokens = {k : v.to(device) for k, v in masked_input_tokens.items() if k != 'token_type_ids'}
            output = model(**masked_input_tokens, token_type_ids = None)
        logits = output.logits 
        predicted_tokens = torch.argmax(logits, dim= -1)

        input_tokens['input_ids'][0, token_indices_to_mask] = predicted_tokens#[0, token_indices_to_mask]

        augmented_text = tokenizer.decode(input_tokens['input_ids'][0], skip_special_tokens=True)
        augmented_sentences.append(augmented_text)
    
    return augmented_sentences

In [164]:
rawdata[rawdata.label == 0].shape[0]

582

In [172]:
num_label = 3
aug_target = rawdata[rawdata.label == 0].shape[0] / rawdata[rawdata.label == num_label].shape[0]
label_3_df = rawdata[rawdata['label'] == num_label]
num_aug =int(np.floor(aug_target))

In [160]:
aug_data = pd.DataFrame(columns = ['text','label'])

In [173]:
for index, row in label_3_df.iterrows():
    augmented_texts = augment_sentence(row['text'], model, tokenizer, num_augmentations= num_aug)
    print(index, row['text'])
    for text in augmented_texts:
        aug_data = aug_data.append({'text': text, 'label' : num_label}, ignore_index = True)

10 편집의 정석 시시각각 변화하는 출판계의 상황과 별개로 글로 쓰인 원고 가 한 권의 책 으로 탄생하기까지 편집 과정의 불변의 진리를 보여주는 우리 시대의 고전. 편집의 정석 Editors on Editing 1962, 1985, 1993 은 1962년에 초판이 출간된 이래 현재까지 편집자, 편집자 지망생, 특히 출판 과정을 알고자 하는 작가에게 없어서는 안 될 귀한 지침서가 되어온 책이다. 집필에 참여한 이들은 사이먼 앤드 슈스터, 바이킹, 헨리 홀츠, 크라운, 호튼 미플린, 하퍼콜린스, 세인트 마틴스 프레스, 옥스퍼드 대학교 출판부, 펭귄USA 등 미국 유수의 출판사에서 편집자로 성장해온 전설적인 편집자, 에이전트, 발행인 39명이다. 각 분야별 최고의 편집자들이 나누는 이야기에는 편집자의 책임과 의무, 원고의 기획과 발굴에서 분야별 세부 편집 기술까지 망라되어 있다. 이 밖에도 편집자가 필요한가 편집의 윤리적, 도덕적 측면 등 도발적이며 때로는 논란을 불러일으키는 다양한 글과 출판과 편집에 관한 참고문헌 목록이 수록되어 있다.
120 상처받은 나를 위한 애도 수업 전 삼성서울병원 정신건강의학과 전문의이자, 자아 심리학의 본거지인 뉴욕에서 프로이트와 정신분석학을 깊게 공부한 저자 강은호 원장은 자신을 정말 사랑하기 위해 애도 가 필요하다고 말한다. 애도는 타인을 떠나보낸 사람들에게만 필요한 것은 아니다. 저자는 시간의 흐름, 우리가 나이를 먹는 것, 소중하게 생각하는 사람들에 대한 실망, 심지어 이루지 못한 소망들까지 모두 상실이라고 말한다. 즉 우리의 삶은 상실의 연속이다. 삶이 계속되는 상실이라면, 애도 역시 우리가 사는 동안 멈출 수 없는 작업이다. 저자는 애도의 4단계 부정, 분노, 슬픔, 수용 를 통해 나를 괴롭히는 부정적인 감정과 작별하고, 내 안의 상처를 다시 살피도록 돕는다. 이 책에서 말하는 애도의 4단계는 어떤 순서가 아니라 상실을 마주한 사람들이 머무르는 곳일 뿐이다. 단계에 얽매이지 않고 자신이 지금 어떤 마음인지를 이해하는 것으로 

KeyboardInterrupt: 

In [150]:
from tqdm import tqdm

In [175]:
aug_data

,text,label
0,한국 창작 SF의 거의 모든 것 none,0
1,애니메이션의 환상적 소환 연구 none [unused1],0
2,와일드 카드. 2 none [unused1],0
3,왕좌의 게임 공식 컬러링북 none [unused1],0
4,SF는 정말 끝내주는데 에이플랫 시리즈 12 none [unused1],0
5,마인크래프트 수수께끼의 수중 도시 양장본 HardCover none [unused1],0
6,마인크래프트 엔더월드의 최후 양장본 HardCover none [unused1],0
7,시사 저널 none,0
8,우리는 누구나 정말로 어찌할 바를 모르고 있다 none,0
9,집으로부터 일만 광년 none [unused1],0


In [179]:
aug_data = pd.DataFrame(columns = ['text','label'])
for num_label in tqdm(range(len(cate2idx))):
    label_df = rawdata[rawdata['label'] == num_label]
    aug_target = rawdata[rawdata.label == 0].shape[0] / rawdata[rawdata.label == num_label].shape[0]
    num_aug =int(np.floor(aug_target))
    for index, row in label_df.iterrows():
        augmented_texts = augment_sentence(row['text'], model, tokenizer, num_augmentations= num_aug)
        for text in tqdm(augmented_texts):
            aug_data = aug_data.append({'text': text, 'label' : num_label}, ignore_index = True)
            break
        break
    break

  0%|                                                                                          | 0/201 [00:00<?, ?it/s]


In [177]:
sentence

'편집의 정석 시시각각 변화하는 출판계의 상황과 별개로 글로 쓰인 원고 가 한 권의 책 으로 탄생하기까지 편집 과정의 불변의 진리를 보여주는 우리 시대의 고전. 편집의 정석 Editors on Editing 1962, 1985, 1993 은 1962년에 초판이 출간된 이래 현재까지 편집자, 편집자 지망생, 특히 출판 과정을 알고자 하는 작가에게 없어서는 안 될 귀한 지침서가 되어온 책이다. 집필에 참여한 이들은 사이먼 앤드 슈스터, 바이킹, 헨리 홀츠, 크라운, 호튼 미플린, 하퍼콜린스, 세인트 마틴스 프레스, 옥스퍼드 대학교 출판부, 펭귄USA 등 미국 유수의 출판사에서 편집자로 성장해온 전설적인 편집자, 에이전트, 발행인 39명이다. 각 분야별 최고의 편집자들이 나누는 이야기에는 편집자의 책임과 의무, 원고의 기획과 발굴에서 분야별 세부 편집 기술까지 망라되어 있다. 이 밖에도 편집자가 필요한가 편집의 윤리적, 도덕적 측면 등 도발적이며 때로는 논란을 불러일으키는 다양한 글과 출판과 편집에 관한 참고문헌 목록이 수록되어 있다.'

In [176]:
aug_data

,text,label
0,한국 창작 SF의 거의 모든 것 none,0
1,애니메이션의 환상적 소환 연구 none [unused1],0
2,와일드 카드. 2 none [unused1],0
3,왕좌의 게임 공식 컬러링북 none [unused1],0
4,SF는 정말 끝내주는데 에이플랫 시리즈 12 none [unused1],0
5,마인크래프트 수수께끼의 수중 도시 양장본 HardCover none [unused1],0
6,마인크래프트 엔더월드의 최후 양장본 HardCover none [unused1],0
7,시사 저널 none,0
8,우리는 누구나 정말로 어찌할 바를 모르고 있다 none,0
9,집으로부터 일만 광년 none [unused1],0


In [6]:
df.head()

,cnt,kind,id,selfLink,title,authors,publisher,publishedDate,description,printType,categories
0,0,books#volume,bRVxvgAACAAJ,https://www.googleapis.com/books/v1/volumes/bR...,한국 창작 SF의 거의 모든 것,박상준외,none,2016-11-05,none,BOOK,none
1,0,books#volume,VXhLDwAAQBAJ,https://www.googleapis.com/books/v1/volumes/VX...,상상력을 자극하는 판타지 SF 스토리북,최종수,행복리뷰_Haengbok Review,2017-11-15,"이 책은 3,000페이지에 달하는 총 40편의 판타지 SF 명작을 담은 스토리북입니...",BOOK,Fiction
2,0,books#volume,BVvIBgAAQBAJ,https://www.googleapis.com/books/v1/volumes/BV...,SF 명예의 전당 3,로버트 하인라인 등,오멜라스,2011-11-25,『SF 명예의 전당 3 : 유니버스』는 미국 SF작가협회(SFWA)에서 펴낸 『Th...,BOOK,Fiction
3,0,books#volume,viIgEAAAQBAJ,https://www.googleapis.com/books/v1/volumes/vi...,외계 생명체의 증언,데이먼 나이트,위즈덤커넥트,2021-02-26,"<책 소개> 우주선 안에서의 살인 사건, 그리고 유일한 목격자가 외계 생명체라는 S...",BOOK,Fiction
4,0,books#volume,T1vIBgAAQBAJ,https://www.googleapis.com/books/v1/volumes/T1...,SF 명예의 전당 4,존 W. 캠벨 등,오멜라스,2011-11-25,『SF 명예의 전당 3 : 유니버스』는 미국 SF작가협회(SFWA)에서 펴낸 『Th...,BOOK,Fiction
